بسم الله الرحمن الرحيم


On a réussi à prendre les livres de fiqh maliki, maintenant on va essayer de mettre l'auteur...

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'products')
#s = s.find_all('div', class_ = 'products product_list row grid')
#lines = s.find('h3', class_ = 's_title_block flex_child')
#Affichage du contenu de la page
#print(lines)

#Extraire les liens
urls = []
for line in s.findAll('a'):
    urls.append(line.get('href'))#line.text.split(\'n') est une liste de hcaine de caractères, donc [0] est 
    #seulement le str.
    #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n

Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.

In [3]:
#Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
urls = list(np.unique(urls))
#urls

On va supprimer les liens qui ne renvoient pas vers des livres : 
 - Ceux qui finissent pas '.asc' ou 'desc
 - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
 - les "javascript"

In [6]:
#Suppression des liens ne renvoyant pas vers des livres.
for link in urls:
    if link.endswith('asc') or link.endswith('desc'):
        try:
            urls.remove(link)
        except ValueError:
            pass
    if link.startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?'):
        urls.remove(link)
urls.remove('javascript:')
urls.remove('javascript:;')
urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')


ValueError: list.remove(x): x not in list

In [5]:
urls

['https://www.sifatusafwa.com/fr/fiqh-ahkam/charh-bidayah-al-mujtahid-wa-nihayah-al-muqtasid-abdullah-al-abadi.html',
 'https://www.sifatusafwa.com/fr/fiqh-ahkam/tabsirah-al-hukkam-fi-usul-al-aqdhiyyah-ibn-farhun-799h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.name.asc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.name.desc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.position.asc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.price.asc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.price.desc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.quantity.desc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?order=product.random.desc',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?page=12',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?page=2',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/?page=3',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-

On va maintenant essayer de regrouper les données des liens dans un dataframe.

## Essai pour le premier lien

In [ ]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))

In [ ]:
L_string

On essaye d'unifier pour enlever les deux "" et afficher en colonnes:

In [ ]:
X = []
Y = []
for element in L_string:
    
    templist = list(np.unique(element))
    templist.remove('')
    X.append(templist[0])
    Y.append(templist[1])

In [ ]:
pd.DataFrame({'c': X, "d": Y})

On remarque qu'il récupère pas les données dans l'ordre, on aimerait avoir une condition sur des nombres tq:

pas 0, pas 1, 2, 3, pas 4, pas 5, 6, 7, pas 8, pas 9, 10, 11, pas 12, 13 

on remarque que la condition qu'il faut pour les différencier : les nombres qui valent 0 ou 1 modulo 4. (8%4 = 0, 9%4 = 1...) mais ca marche pas pour les autres.

In [ ]:
L_string.

In [ ]:
X = []
Y = []
for i in range(len(L_string)):
    templist = list(np.unique(L_string[i]))
    templist.remove('')
    if i%4 == 0 or i%4 ==1:
        X.append(templist[0])
        Y.append(templist[1])
    else:
        X.append(templist[1])
        Y.append(templist[0])


In [ ]:
df = pd.DataFrame({'c': X, "d": Y})

In [ ]:
df.d[0]

In [ ]:
pd.DataFrame({"caracteristiques":X, "test":Y})

In [ ]:
str(list(s)[0]).split('<')

In [ ]:
L_string

On a une liste avec les principales informations d'un livre, on va essayer d'enlever les moins importantes et de garder les plus importantes.

In [ ]:
L_string = list(np.unique(L_string))
try:
    L_string.remove('')
except ValueError:

    X = []
    Y = []
    for i in range(len(L_string)):
        if i%2==0:
            X.append(L_string[i])
        else:
            Y.append(L_string[i])
    df = pd.DataFrame({'test':pd.Series(X), 'test2':pd.Series(Y)})

In [ ]:
L_string